In [119]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests as req
import requests

from urllib.request import urlopen
import re

import time
import sys

In [178]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### Getting books and reviews directly from goodreads.

In [116]:
from importlib import reload  
import src.helper_functions as hf 
hf = reload(hf)

In [4]:
# b_links = []

# page = pages[25]
# r = requests.get(page)
# soup = bs(r.content, 'html')
# divs = soup.find_all('div', {"class": "elementList"})[:50]
# links_for_books = ['http://goodreads.com/' + divs[i].find_all("a", {'class':"bookTitle"})[0].get('href') for i in range(50)]
# b_links.extend(links_for_books)


In [117]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science?page=1'
# pages = [f'https://www.goodreads.com/shelf/show/popular-science?page={x}' for x in range(1, 26)]

In [6]:
# popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science'
# book_links_df = scrap_book_links(popular_science_link)

In [142]:
# Download of book links.
book_links_df = pd.read_csv('../data/book_links_goodreads.csv')
book_links = list(book_links_df['0'])
book_links_copy = book_links.copy()

# removing author links from book_links list.
for book_link in book_links.copy():
    if 'author' in book_link:
        book_links_copy.remove(book_link)

book_links = book_links_copy.copy()

print(f'Total number of books to web-scrap: {len(book_links)}')

Total number of books to web-scrap: 1250


# Get information about the books

In [148]:
def gr_sign_in(driver):
    login_page = 'https://www.goodreads.com'
    driver.get(login_page)

    username = driver.find_element_by_id("userSignInFormEmail")
    username.clear()
    username.send_keys("dilkinn@gmail.com")
    time.sleep(2)

    password = driver.find_element_by_name("user[password]")
    password.clear()
    password.send_keys("pa4a4ok73")
    time.sleep(2)

    driver.find_element_by_xpath("//input[contains(@value,'Sign in')]").click()
    time.sleep(2)
    return driver

## book_links

def scrap_book_links(page1):
    driver = webdriver.Chrome('chromedriver')
    driver = gr_sign_in(driver)
    arr_all = []
    
    driver.get(page1)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_book_links_from_soup(soup)
    arr_all.extend(arr)
    
    while True:
        try:
            print("next_page")
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            time.sleep(1)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_book_links_from_soup(soup)
            arr_all.extend(arr)           
        except:
            driver.close()
            break
            
    print(len(arr_all))
    np.array(arr_all)
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/book_links_goodreads.csv')
    return df

def get_book_links_from_soup(soup):
    b_links = []
    divs = soup.find_all('a', {"class": "leftAlignedImage"})
    for div in divs:
        b_links.append('http://goodreads.com' + div.get('href'))
    return b_links

In [202]:
def get_the_soup(book_link):
    driver.get(book_link)
    time.sleep(5)
    soup = bs(driver.page_source, 'html.parser')
    print('soup from link ok')
    return soup

def get_isbn(soup):
    isbn_str = soup.find_all('div',{"class":"infoBoxRowItem"})[1].get_text()
    isbn_str = isbn_str.replace('\n', '')
    isbn_str = isbn_str.replace('(', '')
    isbn_str = isbn_str.replace(')', '')
    isbn, isbn13 = isbn_str.strip().split()[0], isbn_str.strip().split()[2]
    print('isbn ok')
    return isbn, isbn13

def get_title(soup):
    title_str = soup.find_all('h1',{"id":"bookTitle"})[0].get_text()
    title_str = title_str.replace('\n', "").strip()
    print('title ok')
    return title_str

def get_avg_rating(soup):
    avg_rating = soup.find_all('span',{"itemprop":"ratingValue"})[0].get_text()
    avg_rating = avg_rating.replace('\n', '').strip()
    avg_rating = float(avg_rating)
    print('avg rating ok')
    return avg_rating

def get_author_name(soup):
    author_name = []
    spans = soup.find_all('a',{"class":"authorName"})[0].find_all('span',{'itemprop':'name'})
    for span in spans:
        author_name.append(span.get_text())
    print('author name ok')
    return author_name

def get_book_desc(soup):
    book_desc = soup.find_all('div',{"id":"description"})[0].find_all('span')[1].get_text()
    print('book description ok')
    return book_desc

def get_genres(soup):
    divs = soup.find_all('a',{"class":"actionLinkLite bookPageGenreLink"})
    genres = []
    for div in divs:
        genres.append(div.get_text())
    print('genres ok')
    return genres

def get_book_info(book_link, soup):
    try:
        isbn, isbn13 = get_isbn(soup)
    except:
        print('********** get_book_info: isbn failed **********')
        param = 'isbn'
        try:
            isbn, isbn13 = try_another_edition(soup, param)
        except:
            print('another edition failed')
            isbn, isbn13 = '0', '0'

    try:
        title_str = get_title(soup) 
    except:
        print('********** get_book_info: title failed **********')
        param = 'book_title'
        try:
            title_str = try_another_edition(soup, param)
        except:
            print('another edition failed')
            title_str = '0'
    
    try:
        avg_rating = get_avg_rating(soup)
    except:
        print('********** get_book_info: rating failed **********')
        param = 'avg_rating'
        try:
            avg_rating = try_another_edition(soup, param)
        except:
            print('another edition failed')
            avg_rating = 0

    author_name = []
    try:
        author_name.extend(get_author_name(soup))
    except:
        print('********** author names failed **********')
        param = 'author_name'
        try:
            author_name.extend(try_another_edition(soup, param))
        except:
            print('another edition failed')
              
    try:
        desc = get_book_desc(soup)
    except:
        print('********** book description failed **********')
        param = 'book_desc'
        try:
            desc = try_another_edition(soup, param)
        except:
            desc = '0'

    try:
        print('trying genres')
        genres = get_genres(soup)
    except:
        print('********** genres failed **********')
        param = 'genres'
        try:
            genres = try_another_edition(soup, param)
        except:
            genres = []
  
    dct = {'book_link': book_link, 'isbn': isbn, 'isbn13': isbn13, 
          'book_title' : title_str, 'avg_rating' : avg_rating, 
          'author_name': author_name, 'book_desc': desc, 'genres': genres}
    
    print('get book info ok')
    return dct

In [203]:
def try_another_edition(soup, param):
    dct ={}
    divs = soup.find_all('div',{"class":"otherEdition"})
    links = []
    for div in divs:
        link = div.find('a').get('href')
        links.append(link)
        
    for link in links:
        try:
            soup = get_the_soup(link)
            if param == 'isbn':
                isbn, isbn13 = get_isbn(soup)
                if len(isbn) == 10:
                    print('another edition isbn ok')
                    return isbn, isbn13
            elif param == 'book_title':
                title = get_title(soup)
                if len(title) > 0:
                    print('another edition title ok')
                    return title
            elif param == 'avg_rating':
                avg_rating = get_avg_rating(soup)
                print('another edition avg_rating ok')
                return avg_rating
            elif param == 'author_name':
                author_name = get_author_name(soup)
                print('another edition author_name ok')
                return author_name
            elif param == 'book_desc':
                desc = get_book_desc(soup)
                print('another edition book_desc ok')
                return desc
            elif param == 'genres':
                genres = get_genres(soup)
                print('another edition genres ok')
                return genres
        except:
            continue

In [199]:
def get_reviews_array_from_soup(soup):
    arr = []
    divs = soup.find_all('div',{"id":"bookReviews"})[0].find_all('div',{'class':'friendReviews elementListBrown'})
    
    for div in divs:
        try:
            user_link = 'http://goodreads.com' + div.find_all('span',{'itemprop':'author'})[0].find('a').get('href')
        except:
            print('No user link')
            user_link = '0'
        try:
            ranking = len(div.find_all('span', {'class':'staticStar p10'}))
        except:
            print('No ranking')
            ranking = '0'
        try:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[1].get_text()
        except:
            try:
                review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[0].get_text()
            except:
                review = '0'
                
        dct = {"isbn":isbn, 'user_link': user_link, 'ranking':ranking, "review": review}
        arr.append(dct)
    print(f'get {len(arr)} reviews ok')
    return arr

def get_reviews_from_book_link(isbn, b_l):
    arr_all = []
    
    driver.get(b_l)
    time.sleep(10)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    arr = get_reviews_array_from_soup(soup)
    arr_all.extend(arr)
    
    for k in range(1,10):
        try:
            # click next page:
            driver.find_element_by_xpath("//a[contains(@rel,'next')]").click()
            # wait for URL to change with 15 seconds timeout
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "reviews")))
            time.sleep(3)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            arr = get_reviews_array_from_soup(soup)
            arr_all.extend(arr)         
        except:
            try:
                time.sleep(15)
                html = driver.page_source
                soup = bs(html, 'html.parser')
                arr = get_reviews_array_from_soup(soup)
                arr_all.extend(arr)
                print(f'Exception: page {k+1} ok')
            except:
                print(f'Exception: no reviews on page {k+1}!!!!!!!')
    
    print(f'got {len(arr_all)} reviews for this book')
    df = pd.DataFrame(arr_all)
    df.to_csv(f'../data/revs_{isbn}.csv')
    print('reviews file created')
    return df

In [204]:
# Open the web browser and log in to goodreads:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

In [ ]:
# Script to scrap the books

# Initializing the lists:
books_arr = []
failed_books_list = []

for n in range(0, 1250, 50):
    chunk_arr = []
    for i, book_link in enumerate(book_links[n:n+50]):
        sys.stdout.write(f'book n : {i}\n')
        try:
            #Getting the soup:
            soup = get_the_soup(book_link)
            dct = get_book_info(book_link, soup)
            isbn = dct['isbn']
            reviews = get_reviews_from_book_link(isbn, book_link)
            chunk_arr.append(dct)
        except:
            failed_books_list.append(book_l)
            print('!!!!!!!!!!!!!!!!!!! BOOK FAILED !!!!!!!!!!!!!!!!!!')
    df = pd.DataFrame(chunk_arr)
    df.to_csv(f'../data/gr_books_df_{n}.csv')
    books_arr.extend(chunk_arr)

all_books_df = pd.DataFrame(books_arr)
all_books_df.to_csv(f'../data/all_books_df.csv') 

book n : 0
soup from link ok
isbn ok
title ok
avg rating ok
author name ok
book description ok
trying genres
genres ok
get book info ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
got 300 reviews for this book
reviews file created
book n : 1
soup from link ok
isbn ok
title ok
avg rating ok
author name ok
book description ok
trying genres
genres ok
get book info ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
get 30 reviews ok
Exception: page 6 ok
get 30 reviews ok
Exception: page 7 ok
get 30 reviews ok
Exception: page 8 ok
get 30 reviews ok
Exception: page 9 ok
get 30 reviews ok
Exception: page 10 ok
got 300 reviews for this book
reviews file created
book n : 2
soup from link ok
isbn ok
title ok
avg rating ok
author name ok
book description ok
trying genres
genres ok
get book info ok
get 30 reviews ok
get 30

In [112]:
new_fail_list = []
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

n=1200
for i, book_l in enumerate(failed_books_list):
    sys.stdout.write("\r" + f'book n : {i}\n')
    sys.stdout.flush()
    try:
        soup = get_the_soup(book_l)
        dct = get_book_info(book_l, soup)
        isbn = dct['isbn']
        desc = dct['book_description']
        book_title = dct['book_title']
        avg_rating = dct['avg_rating']
        if (len(isbn) != 10) or (len(desc) == 0) or (len(book_title) == 0) or (avg_rating == 0):
            print(f'isbn is broken : {isbn}')
            try_another_edition(soup)
        else:
            new_fail_list.append(bool_l)
    except:
        new_fail_list.append(book_l)
        continue

book n : 0
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn is broken : 0
https://www.goodreads.com/book/show/20873740-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/23198201-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/23346740-sapiens
soup ok
get_book_info: isbn failed
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
isbn fail
https://www.goodreads.com/book/show/20839196-sapiens
soup ok
isbn ok
title ok
original title ok
avg rating ok
author_link ok
author names ok
book description ok
get book info ok
success
get reviews array f

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = gr_sign_in(driver)

In [169]:
soup = get_the_soup(book_links[3])
# dct = get_book_info(book_l, soup)
# isbn = dct['isbn']

soup ok


['Nonfiction',
 'History',
 'Science',
 'Philosophy',
 'Anthropology',
 'Audiobook',
 'Sociology',
 'Historical',
 'Psychology',
 'Biology',
 'Evolution']

In [174]:
current_url = driver.current_url